In [1]:
import pandas as pd
import numpy as np
import requests
import time
import json
import pprint
import os
import datetime
from random import sample
from sqlalchemy import create_engine

In [2]:
#pull in data and get rid of non-province entries
#get rid of america as a province

winemag2 = "./csv/winemag-data-130k.csv"
winemag_data2 = pd.read_csv(winemag2)
#df = df[df.line_race != 0]
winemag_data2 = winemag_data2[winemag_data2.province != 'America']

In [3]:
#remove unnecessary/unused columns
winemag2_df = winemag_data2.drop(columns=["Unnamed: 0","description","region_1","region_2","taster_name","taster_twitter_handle","title"])
winemag2_df.head()

,country,designation,points,price,province,variety,winery
0,Italy,Vulkà Bianco,87,NaN,Sicily & Sardinia,White Blend,Nicosia
1,Portugal,Avidagos,87,15.0,Douro,Portuguese Red,Quinta dos Avidagos
2,US,NaN,87,14.0,Oregon,Pinot Gris,Rainstorm
3,US,Reserve Late Harvest,87,13.0,Michigan,Riesling,St. Julian
4,US,Vintner's Reserve Wild Child Block,87,65.0,Oregon,Pinot Noir,Sweet Cheeks


In [4]:
#check row counts
winemag_all_rows = winemag2_df
winemag_all_rows.count()

country        129813
designation     92437
points         129876
price          120880
province       129813
variety        129875
winery         129876
dtype: int64

In [5]:
# have a look at the country counts and averages
by_country_count = winemag_all_rows.groupby("country").count()
by_country_count
by_country_mean = winemag_all_rows.groupby("country").mean().reset_index() #get country to show
by_country_mean.head()


,country,points,price
0,Argentina,86.710263,24.510117
1,Armenia,87.500000,14.500000
2,Australia,88.580507,35.437663
3,Austria,90.101345,30.762772
4,Bosnia and Herzegovina,86.500000,12.500000


In [17]:
winemag_all_rows.head()

,country,designation,points,price,province,variety,winery
0,Italy,Vulkà Bianco,87,NaN,Sicily & Sardinia,White Blend,Nicosia
1,Portugal,Avidagos,87,15.0,Douro,Portuguese Red,Quinta dos Avidagos
2,US,NaN,87,14.0,Oregon,Pinot Gris,Rainstorm
3,US,Reserve Late Harvest,87,13.0,Michigan,Riesling,St. Julian
4,US,Vintner's Reserve Wild Child Block,87,65.0,Oregon,Pinot Noir,Sweet Cheeks


In [19]:
winemag_all_rows.index.name = 'id'
winemag_all_rows.head()

,country,designation,points,price,province,variety,winery
id,,,,,,,
0,Italy,Vulkà Bianco,87,NaN,Sicily & Sardinia,White Blend,Nicosia
1,Portugal,Avidagos,87,15.0,Douro,Portuguese Red,Quinta dos Avidagos
2,US,NaN,87,14.0,Oregon,Pinot Gris,Rainstorm
3,US,Reserve Late Harvest,87,13.0,Michigan,Riesling,St. Julian
4,US,Vintner's Reserve Wild Child Block,87,65.0,Oregon,Pinot Noir,Sweet Cheeks


In [20]:
# just in case I want to look at the data 
winemag_all_rows.to_csv("./csv/winemag_cleaned.csv")

In [21]:
connection_string = "postgres:Issimo86*@localhost:5432/etl_project"
engine = create_engine(f'postgresql://{connection_string}')

In [24]:
engine.execute('CREATE TABLE "winedata" ('
'id INTEGER PRIMARY KEY,'
'country varchar,'
'designation varchar,'
'points int,'
'price float,'
'province varchar,'
'variety varchar,'
'winery varchar'
');')

ProgrammingError: (psycopg2.errors.DuplicateTable) relation "winedata" already exists

[SQL: CREATE TABLE "winedata" (id INTEGER PRIMARY KEY,country varchar,designation varchar,points int,price float,province varchar,variety varchar,winery varchar);]
(Background on this error at: http://sqlalche.me/e/f405)

In [10]:
engine.table_names()

['winedata']

In [23]:
winemag_all_rows.to_sql(name='winedata', con=engine, if_exists='append', index=True)

In [25]:
engine.execute('SELECT * FROM winedata LIMIT 5').fetchall()

[(0, 'Italy', 'Vulkà Bianco', 87, None, 'Sicily & Sardinia', 'White Blend', 'Nicosia'),
 (1, 'Portugal', 'Avidagos', 87, 15.0, 'Douro', 'Portuguese Red', 'Quinta dos Avidagos'),
 (2, 'US', None, 87, 14.0, 'Oregon', 'Pinot Gris', 'Rainstorm'),
 (3, 'US', 'Reserve Late Harvest', 87, 13.0, 'Michigan', 'Riesling', 'St. Julian'),
 (4, 'US', "Vintner's Reserve Wild Child Block", 87, 65.0, 'Oregon', 'Pinot Noir', 'Sweet Cheeks')]